In [ ]:
# default_exp model
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload python modules if there are changes in the
%autoreload 2

ModuleNotFoundError: No module named 'lab_black'

# Loss

***input:*** clean and tidy dataset from data notebook + ML / simulation / analytics python module from hypotheses space notebook

***output:*** evaluation of the methods, fitted hyperparameters of the best model(s)

***description:***

In this notebook you define evaluation metrics (loss) of the ML methods implemented, and evaluate them fitting the models to the whole dataset.

In [ ]:
#hide
from nbdev.showdoc import *

## Import relevant modules

## Load the clean and tidy data produced in the data notebook

## Split the data into training, testing and validation data

by default first separate validation dataset with stratified split, then use stratified k-fold to extract training and testing datasets

## Define Loss function

## Train and evaluate the models

## Visualize results

## Select validate best model (if hyperparameters are optimized)

## Visualize validation

## Conclusions

Explain the model behaviour results to your best understanding

compare to benchmark results

## Save results, including fitted models and hyperparameters for production use
these are output of of the notebook